Code to download data for each property URL on Redfin's website.

In [3]:
import pyautogui
import random
import time
import pyperclip
import math
import xerox
import subprocess
import pandas as pd

def getClipboardData():
    task = subprocess.Popen(['pbpaste'],stdout=subprocess.PIPE, close_fds=True)
    stdout, stderr = task.communicate()
    return(stdout.decode('utf-8'))

def move_naturally(end_x, end_y, duration=None):
    """
    Move mouse naturally to target position using bezier curve movement
    
    Args:
        end_x (int): Target X coordinate
        end_y (int): Target Y coordinate
        duration (float, optional): Time to complete movement. If None, calculated based on distance
    """
    # Ensure pyautogui doesn't instantly kill the program if mouse goes to corner
    pyautogui.FAILSAFE = True
    
    # Get start position
    start_x, start_y = pyautogui.position()
    
    # Calculate distance
    distance = math.sqrt((end_x - start_x)**2 + (end_y - start_y)**2)
    
    # Set duration based on distance if not specified
    if duration is None:
        duration = distance / 1500  # Adjust this value to change speed
        
    # Generate control points for bezier curve
    control_x1 = start_x + random.randint(0, int(distance/2))
    control_y1 = start_y + random.randint(-int(distance/2), int(distance/2))
    control_x2 = end_x - random.randint(0, int(distance/2))
    control_y2 = end_y + random.randint(-int(distance/2), int(distance/2))
    
    # Number of steps
    steps = int(duration * 50)  # 50 movements per second
    
    # Move through bezier curve points
    for i in range(steps + 1):
        t = i / steps
        
        # Bezier curve formula
        x = (1-t)**3 * start_x + 3*(1-t)**2 * t * control_x1 + \
            3*(1-t) * t**2 * control_x2 + t**3 * end_x
        y = (1-t)**3 * start_y + 3*(1-t)**2 * t * control_y1 + \
            3*(1-t) * t**2 * control_y2 + t**3 * end_y
        
        # Move to the next point
        pyautogui.moveTo(int(x), int(y))
        
        # Add small random delay
        time.sleep(duration/steps + random.uniform(0.0001, 0.001))

def scroll_naturally(total_amount, duration=None):
    """
    Scroll naturally with variable speed and random pauses
    
    Args:
        total_amount: Positive for scroll up, negative for scroll down
        duration: Approximate time to complete scroll (optional)
    """
    # Set default duration based on scroll amount if not specified
    if duration is None:
        duration = abs(total_amount) / 500  # Adjust this divisor to change default speed
    
    # Initialize variables
    scrolled = 0
    start_time = time.time()
    
    while scrolled < abs(total_amount):
        # Calculate remaining amount
        remaining = abs(total_amount) - scrolled
        
        # Variable scroll speed
        scroll_step = random.randint(
            min(4, remaining),  # Minimum scroll amount
            min(16, remaining)  # Maximum scroll amount
        )
        
        # Determine scroll direction
        if total_amount < 0:
            scroll_step = -scroll_step
            
        # Perform scroll
        pyautogui.scroll(scroll_step)
        
        # Add human-like pause
        time.sleep(random.uniform(0.05, 0.15))
        
        # Sometimes add a longer pause
        if random.random() < 0.1:  # 10% chance
            time.sleep(random.uniform(0.2, 0.5))
            
        scrolled += abs(scroll_step)
        
        # Check if we're taking too long
        if time.time() - start_time > duration * 1.5:
            # Finish the scroll faster
            pyautogui.scroll(total_amount - scrolled)
            break

def write_to_file_in_append_mode(file_name, data):
    try:
        # Open the file in append mode ('a')
        with open(file_name, 'a') as file:
            # Write the data to the file, adding a newline after each entry
            file.write(data + "\n")
    except Exception as e:
        print(f"Error writing to file: {e}")

def scrape_url(url):

    js = """

    function arrayToSnakeCaseJson(array) {
        return array.reduce((acc, item) => {
            const [key, value] = item.split(':').map(str => str.trim());
            if (key.toLowerCase().includes("has")) {
            acc[key.replace(/[\s.:]+/g, '_').replace(/([a-z])([A-Z])/g, '$1_$2').toLowerCase()] = true
            } else {
            acc[key.replace(/[\s.:]+/g, '_').replace(/([a-z])([A-Z])/g, '$1_$2').toLowerCase()] = value;
            }
            return acc;
        }, {});
    }

    let mainObject = {}; // Consolidate all data into a single object
    for (let i = 0; i < 6; i++) {
        let section = document.getElementsByClassName("amenities-container")[0].children[i];
        if (section) {
            mainObject = Object.assign({}, arrayToSnakeCaseJson(section.innerText.split("\\n")), mainObject)
        }
    }
    
        // Add Additional Data
        mainObject['date_sold'] = document.querySelector('[data-rf-test-id=home-sash]')?.textContent?.split("SOLD")[1]?.trim() || "NA";
        mainObject['address'] = document.querySelector('[data-rf-test-id=abp-homeinfo-homeaddress]')?.children[0]?.textContent || "NA";
        mainObject['estimated_sales_range'] = document.getElementsByClassName("sale-price-range")[0]?.innerText || "NA";
        mainObject['estimated_rental_value'] = document.getElementsByClassName("estimate")[0]?.innerText || "NA";
        mainObject['estimated_monthly_cost'] = document.getElementsByClassName("CostOfOwnershipSectionContent")[0]?.children[0]?.children[0]?.children[0]?.innerText?.split(" ")[0] || "NA";
        mainObject['monthly_mortgage_payment'] = document.getElementsByClassName("CostOfOwnershipSectionContent")[0]?.children[0]?.children[2]?.children[0]?.innerText?.split("\\n")[1]?.trim() || "NA";
        mainObject['estimated_market_value'] = document.querySelector('[data-rf-test-id=abp-price]')?.children[0]?.children[0]?.innerText || "NA";
        mainObject['price_per_sqft'] = document.querySelector('[data-rf-test-id=house-info]')?.children[2]?.children[0]?.children[0]?.children[0]?.children[0]?.innerText?.split("\\n").find(text => text.includes('Redfin Estimate'))?.split(" ")[0] || "NA";

        // Add Walk Score, Transit Score, and Bike Score
        mainObject['walk_score'] = document.getElementsByClassName("walkscore-pills")[0].children[0]?.innerText?.split("/")[0]?.trim() || "NA";
        mainObject['transit_score'] = document.getElementsByClassName("walkscore-pills")[0].children[1]?.innerText?.split("/")[0]?.trim() || "NA";
        mainObject['bike_score'] = document.getElementsByClassName("walkscore-pills")[0].children[2]?.innerText?.split("/")[0]?.trim() || "NA";
        mainObject
    
"""
#open new tab and close the old one
    move_naturally(330, 58)
    pyautogui.click(x=330, y=58)
    pyautogui.click(x=293, y=57)

#click and type the url
    move_naturally(182, 100)
    pyautogui.click(x=182, y=100)

    pyautogui.write(url)
    pyautogui.press('enter')

    time.sleep(2)

    # Scroll down
    move_naturally(300, 400)
    scroll_naturally(-4000, 1)

    # Right click and click on inspect
    move_naturally(250, 740)
    pyautogui.rightClick(x=250, y=740)
    move_naturally(289, 1087)
    pyautogui.click(x=289, y=1087)

    # Click the console button
    move_naturally(969, 140)
    time.sleep(0.1)
    pyautogui.click(x=969, y=140)
    pyautogui.click(x=969, y=140)

    # Click on clear console
    move_naturally(740, 165)
    pyautogui.click(x=740, y=165)
    pyautogui.click(x=740, y=165)

    # Click the console
    move_naturally(760, 400)
    pyautogui.click(x=760, y=400)

    # Type the js code
    pyperclip.copy(js)
    pyautogui.hotkey('command', 'v')
    pyautogui.press('enter')

    # copy the object
    move_naturally(850, 750)
    pyautogui.rightClick(x=850, y=750)
    move_naturally(910, 760)
    pyautogui.click(x=910, y=760)
    
    # copy the object
    move_naturally(850, 750)
    pyautogui.rightClick(x=850, y=750)
    move_naturally(910, 760)
    pyautogui.click(x=910, y=760)

   # print(str(pyperclip.paste()))
    write_to_file_in_append_mode("data2.txt", str(pyperclip.paste())+",")



In [4]:
target_url_from_excel = pd.read_excel("/Users/anjaliraj/Documents/Capstone/Data_Download/URL_List.xlsx")
len(target_url_from_excel)

7930

In [ ]:
time.sleep(2)
write_to_file_in_append_mode("data3.txt","[")

for i in range(len(target_url_from_excel)):
    url = target_url_from_excel.loc[i,'URL']
    scrape_url(url)

In [25]:
import pandas as pd
import json

# Load the JSON array from the .txt file
with open('data4.js', 'r') as file:
    json_data = json.load(file)  # Parses the JSON array

# Convert the JSON array to a pandas DataFrame
df = pd.DataFrame(json_data)

df.head()


,beds,baths,sq_ft_,stories,lot_size,style,year_built,year_renovated,county,apn,...,carport_spaces,#_of_rooms_(total),#_of_units,room_types,green_water_conservation,green_energy_generation,upper_level_features,has_association_association_name,green_building_verification_type,view
0,3,2.0,"1,193",1.0,"5,980 square feet",Single Family Residential,1953,1953,Santa Clara County,19317008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1.0,974,1.0,"6,300 square feet",Single Family Residential,1952,1952,Santa Clara County,17019032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2.0,"1,389",1.0,"6,100 square feet",Single Family Residential,1956,1956,Santa Clara County,14735026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,,"2,808",2.0,"7,700 square feet",Multi-Family (2-4 Unit),1979,1979,Santa Clara County,16039010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2.0,"1,466",1.0,"5,670 square feet",Single Family Residential,1952,1952,Santa Clara County,17019025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df.to_excel('Full Data_520.xlsx')

In [26]:
df.shape

(520, 300)

In [10]:
import os
import pandas as pd

def append_excel_files(folder_path):
    # Initialize an empty DataFrame
    combined_df = pd.DataFrame()

    # List all Excel files in the folder
    excel_files = [file for file in os.listdir(folder_path) if file.endswith(('.xlsx', '.xls'))]

    # Loop through each Excel file and append to the combined DataFrame
    for file in excel_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)

    return combined_df

# Specify the folder containing the Excel files
folder_path = '/Users/anjaliraj/Documents/Capstone/Final Data'

# Get the combined DataFrame
final_df = append_excel_files(folder_path)

# Save the combined data to a new Excel file (optional)
final_df.to_excel('/Users/anjaliraj/Documents/Capstone/Final Data/Combined_Data.xlsx', index=False)

print("All Excel files have been combined into combined_output.xlsx.")


All Excel files have been combined into combined_output.xlsx.


In [11]:
final_df.shape

(3847, 27)

In [30]:
# Load the merged Excel file
file_path = 'combined_output.xlsx'  # Path to the merged Excel file
df = pd.read_excel(file_path)

# Calculate the count of non-null values for each column
column_data_count = df.count()

# Sort columns by the count of non-null values in ascending order
sorted_columns = column_data_count.sort_values()

# Convert to DataFrame for better presentation (optional)
sorted_columns_df = sorted_columns.reset_index()
sorted_columns_df.columns = ['Column_Name', 'Non_Null_Count']

print("Columns sorted by the least amount of data:")
print(sorted_columns_df)

# Save the result to an Excel file (optional)
sorted_columns_df.to_excel('columns_sorted_by_data.xlsx', index=False)


Columns sorted by the least amount of data:
                               Column_Name  Non_Null_Count
0             has_carport_parking_features               1
1                             lot_location               1
2                    water_heater_features               1
3    garage_is_detached_#_of_garage_spaces               1
4              #_of_parking_spaces_(total)               1
..                                     ...             ...
411                  estimated_sales_range            3026
412                          transit_score            3058
413                             walk_score            3060
414                                address            3060
415                             Unnamed: 0            3060

[416 rows x 2 columns]


In [ ]:
import pandas as pd
from fuzzywuzzy import process

# Define the list of target columns
target_columns = ['Estimated_market_value', 'Selling_Price', 'Date_Sold', 'Estimated_Salesrange',
                  'Estimated_Monthly_Cost', 'Estimated_Rental_Value', 'Beds', 'Bathrooms', 'Square_Feet',
                  'Address', 'Year_Built', 'Price_Per_Sqft', 'Stories', 'Cooling_Type', 'Heating_Type',
                  'Lot_Size', 'Parking_Spaces', 'Garage_Spaces', 'Monthly_Mortgage_Payment', 'Agent_Name',
                  'Laundry', 'Flooring', 'Roof', 'Fencing', 'Walk_Score', 'Transit_Score', 'Bike_Score','Baths']

# Load the merged Excel file
file_path = 'combined_output.xlsx'  # Path to the merged Excel file
df = pd.read_excel(file_path)

# Define a threshold for matching (e.g., 80% similarity)
threshold = 80

# Match columns from the DataFrame to the target columns
columns_to_keep = []
for col in df.columns:
    match, score = process.extractOne(col, target_columns)
    if score >= threshold:
        columns_to_keep.append(col)

# Create a new DataFrame with only the matched columns
filtered_df = df[columns_to_keep]

# Save the filtered DataFrame to a new Excel file (optional)
filtered_df.to_excel('filtered_output.xlsx', index=False)

print(f"Kept columns: {columns_to_keep}")
print(f"Dropped columns: {set(df.columns) - set(columns_to_keep)}")


In [2]:
# List of columns to drop
columns_to_drop = [ 'stories', 'heating', 'lot_size_area_minimum_units', 'lot_size_area_maximum_units', 
                   'lot_size_source', 'bathroom_description', 'lot_size_square_feet', 'lot_size_area', 
                   'lot_size_units', 'lot_size_acres', '#_of_bathrooms_full', '#_of_bathrooms_partial', 
                   'laundry_features', 'has_attached_garage_garage_spaces', 'roof_framing_type', 'acres', 
                   'garage', 'unparsed_address', 'street_address_filtered', '#_of_bathrooms_(full)', 'other_bathroom_features', 
                   'other_bathrooms_included_features', 'primary_bathroom_included_features', '#_of_bathrooms_(total)', 'parking_access', 
                   '#_of_parking_spaces', 'heating_fuel_type', 'roof_covering_type', 'roof_shape_type', 'total_#_of_stories', 'year_built_source', 
                   'has_laundry', 'main_level_bathrooms', '#_of_garage_spaces', 'master_bathroom_features', 'open_parking_spaces', '#_of_bathrooms_(partial)', 
                   'lot_size_dimensions', 'has_laundry_in_garage_', 'attached_garage_#_of_garage_spaces', 'total_stories', 'laundry_location', 'laundry_utilities', 
                   '#_of_garage_parking_spaces', '#_of_non-garage_parking_spaces', 'stories_type', 'has_laundry_gas_&amp;_electric_dryer_hookup_', 'has_laundry_inside_', 
                   'has_laundry_dryer_included_', 'has_laundry_in_closet_', 'garage_is_detached_#_of_garage_spaces', '#_of_parking_spaces_(total)', 'has_laundry_individual_room_', 
                   'rent', 'count_of_bathrooms', '#_of_full_bathrooms', '#_of_half_bathrooms', 'has_laundry_see_remarks_', 'lot_size_(sq_ft_)', 'lot_size_(acres)', 'lot_size_sq_ft_']

# Drop the specified columns from the DataFrame
filtered_df = filtered_df.drop(columns=columns_to_drop, errors='ignore')

# Save the updated DataFrame to a new Excel file (optional)
filtered_df.to_excel('Updated_Filtered_Output.xlsx', index=False)


In [3]:
# Dictionary for renaming columns: {'current_name': 'new_name'}
columns_to_rename = {
    'estimated_market_value': 'Estimated_market_value',
    'date_sold': 'Date_Sold',
    'estimated_sales_range': 'Estimated_Salesrange',
    'estimated_monthly_cost':'Estimated_Monthly_Cost',
    'estimated_rental_value':  'Estimated_Rental_Value',
    'beds':'Beds',  '#_of_bathrooms_total':'Bathrooms',  'square_feet_(estimated)':'Square_Feet',
    'address': 'Address',  'year_built': 'Year_Built',  'price_per_sqft': 'Price_Per_Sqft',
    '#_of_stories':  'Stories',  'cooling':'Cooling_Type',  'heating_type': 'Heating_Type',
    'lot_size': 'Lot_Size',  'parking_spaces': 'Parking_Spaces',  'garage_spaces': 'Garage_Spaces',
    'monthly_mortgage_payment': 'Monthly_Mortgage_Payment',  'laundry': 'Laundry', 
    'flooring':'Flooring',  'roof':'Roof', 'fencing':'Fencing', 'walk_score':'Walk_Score',
    'transit_score':'Transit_Score', 'bike_score':'Bike_Score'
}

# Rename the columns
filtered_df = filtered_df.rename(columns=columns_to_rename)

# Save the updated DataFrame to a new Excel file (optional)
filtered_df.to_excel('Updated_Filtered_Output.xlsx', index=False)